In [1]:
import pandas as pd 
df = pd.read_csv("cc3m_full_with_path.csv")

In [2]:
df

,url,key,status,width,height,exif,caption,alt_txt,path
0,https://ak0.picdn.net/shutterstock/videos/3380...,1470004,success,852.0,480.0,{},the effect of moving the large colored particl...,"""""",/home/temp/Desktop/ccm3_full/00147/001470004.jpg
1,https://ak9.picdn.net/shutterstock/videos/2377...,1470024,success,852.0,480.0,{},wing of an airplane passing over clouds at hig...,"""""",/home/temp/Desktop/ccm3_full/00147/001470024.jpg
2,https://ak7.picdn.net/shutterstock/videos/1292...,1470042,success,852.0,480.0,{},man holding the keys to her new car in a motor...,"""""",/home/temp/Desktop/ccm3_full/00147/001470042.jpg
3,https://ak9.picdn.net/shutterstock/videos/1147...,1470044,success,852.0,480.0,{},background with particles and stars .,"""""",/home/temp/Desktop/ccm3_full/00147/001470044.jpg
4,https://ak8.picdn.net/shutterstock/videos/5176...,1470039,success,852.0,480.0,{},food canned in mason jars on shelves in a stor...,"""""",/home/temp/Desktop/ccm3_full/00147/001470039.jpg
...,...,...,...,...,...,...,...,...,...
2555423,http://l7.alamy.com/zooms/5f24d20339df4077a506...,2199920,success,640.0,447.0,{},senior woman in the park,"""""",/home/temp/Desktop/ccm3_full/00219/002199920.jpg
2555424,http://l7.alamy.com/zooms/7c2ad41134dc4b1984d9...,2199914,success,640.0,446.0,{},remote house in the forest by lake,"""""",/home/temp/Desktop/ccm3_full/00219/002199914.jpg
2555425,http://l7.alamy.com/zooms/0dd49ace792a4db6be39...,2199999,success,640.0,448.0,{},wind blown tree on the beach,"""""",/home/temp/Desktop/ccm3_full/00219/002199999.jpg
2555426,http://l7.alamy.com/zooms/28f1e935be484181a90c...,2199994,success,640.0,447.0,{},snow covered city park in a winter day .,"""""",/home/temp/Desktop/ccm3_full/00219/002199994.jpg


In [9]:
from dataset import * 
from tqdm import tqdm 

In [10]:
from torchvision import datasets, transforms
transform = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
        transforms.Resize((224, 224))
    ])
dataset = CC3M(transform, transform)

In [11]:
from PIL import ImageFont, ImageDraw, Image 

def render_text(txt:str, image_size: int=224, font_size: int = 16, max_chars=768,
                       background_brightness=127, text_brightness=0,
                       lower=True, monospace=False, spacing=1, min_width=4,
                       resize_method="area", max_width=28):
    if len(txt)> max_chars:
        txt = txt[:max_chars]
    if lower: 
        txt = txt.lower() 
    wrapper = textwrap.TextWrapper(width=max_width)
    lines = wrapper.wrap(txt) 
    new_txt = ""
    for line in lines: 
        new_txt+= line+'\n'
    image = Image.new("RGBA", (image_size*3,image_size*3), (background_brightness,background_brightness,background_brightness))
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("/home/temp/Desktop/CLIPPO/unifont-15.0.06.otf", font_size*3)
    draw.text((0, 0), new_txt, (text_brightness,text_brightness,text_brightness), font=font, spacing=spacing)
    img_resized = image.resize((image_size,image_size), Image.ANTIALIAS)
    return img_resized


In [22]:
import torch 
from network import CLIPPO
clippo = CLIPPO()
clippo = clippo.cuda() 
#clippo.load_state_dict(torch.load("/home/temp/Desktop/KD/CLIPPO/clippo.pt"), strict=False) 

In [23]:
image1, text1 = dataset[0]
image2, text2 = dataset[1]
image3, text3 = dataset[2]
image4, text4 = dataset[3]
stext, simage = torch.stack([text1, text2, text3, text4]).cuda(), torch.stack([image1, image2, image3, image4]).cuda()

In [24]:
clippo(simage, stext)

(tensor([[-1.3269, -1.3930],
         [-0.2609, -0.3248]], device='cuda:0', grad_fn=<MmBackward0>),
 tensor([[-1.3269, -0.2609],
         [-1.3930, -0.3248]], device='cuda:0', grad_fn=<TBackward0>))

In [25]:
text_embed = clippo.encoder(torch.stack([text1, text2]).cuda())
image_embed = clippo.encoder(torch.stack([image1, image2]).cuda())

In [29]:
image_embed

tensor([[-1.6836,  0.7750, -1.2643,  ..., -1.1814, -0.7989, -1.3964],
        [-2.3557,  0.9510, -0.9564,  ..., -0.6105, -0.3591, -1.6014]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [30]:
text_embed

tensor([[-0.0419, -0.7034, -2.3115,  ...,  0.3312,  1.3117, -0.5221],
        [-0.1148, -0.7164, -2.2922,  ...,  0.2798,  1.3894, -0.4874]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [31]:
image_features = image_embed / image_embed.norm(dim=1, keepdim=True)
text_features = text_embed / text_embed.norm(dim=1, keepdim=True)

In [32]:
image_features @ text_features.t()

tensor([[-0.0929, -0.0975],
        [-0.0183, -0.0227]], device='cuda:0', grad_fn=<MmBackward0>)

In [43]:
(image_features @ image_features.t()).softmax(dim=0)

tensor([[0.5199, 0.4801],
        [0.4801, 0.5199]], device='cuda:0', grad_fn=<SoftmaxBackward0>)